In [1]:
from langchain_community.llms import HuggingFaceHub
from langchain_community.graphs import Neo4jGraph

In [2]:
from langchain_community.llms import HuggingFaceHub

repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

llm_new = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    huggingfacehub_api_token = "<your-api-key>",
    model_kwargs={
        "max_new_tokens": 100,
        "top_k": 10,
        "top_p": 0.95,
        "typical_p": 0.95,
        "temperature": 0.7,
    },
)

/Users/dhruvchandel/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [3]:
graph = Neo4jGraph(
    url="neo4j+s://46a80122.databases.neo4j.io:7687",
    username="<user>", 
    password="<pass>",
    refresh_schema=True, 
)

In [4]:
schema=graph.schema
print(graph.schema)

Node properties are the following:
Article {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Code {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Dataset {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},SoftwareApplication {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING}
Relationship properties are the following:

The relationships are the following:
(:Article)-[:uses]->(:Dataset),(:Article)-[:provides]->(:Code),(:Article)-[:provides]->(:SoftwareApplication),(:Article)-[:cites]->(:Article)


In [5]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
Examples: Here are a few examples of generated Cypher statements for particular questions:
# How many published paper are there?
MATCH (a:Article)
RETURN COUNT(a) AS numberOfArticles


The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)



In [6]:
question = "How many published papers are there?"

In [7]:

prompt = CYPHER_GENERATION_PROMPT.format(schema=schema, question=question)

In [8]:
prompt

'Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\nNode properties are the following:\nArticle {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Code {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},Dataset {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING},SoftwareApplication {id: INTEGER, name: STRING, url: STRING, creativeWorkType: STRING, metadata_url: STRING}\nRelationship properties are the following:\n\nThe relationships are the following:\n(:Article)-[:uses]->(:Dataset),(:Article)-[:provides]->(:Code),(:Article)-[:provides]->(:SoftwareApplication),(:Article)-[:cites]->(:Article)\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions th

In [9]:
response = llm_new.invoke(prompt)

In [10]:
def extract_cypher_query(output):
    # Split the output into lines
    lines = output.strip().split('\n')
    
    # Find the index of the last line containing the Cypher query
    for i in range(len(lines) - 1, -1, -1):  # Iterate in reverse
        if lines[i].strip().startswith('MATCH'):
            # Join all lines starting from this index to the end
            return '\n'.join(lines[i:])
    
    return None  # Return None if no query found


In [11]:
cypher_query = extract_cypher_query(response)

In [12]:
print(cypher_query)

MATCH (a:Article)
RETURN COUNT(a) AS numberOfArticles


In [13]:
def execute_cypher_query(graph, cypher_query):
    with graph._driver.session() as session:  # Use the session from the existing graph object
        result = session.run(cypher_query)
        return [record.data() for record in result]

In [14]:
print(execute_cypher_query(graph, cypher_query))

[{'numberOfArticles': 30}]


In [15]:
def transform_to_natural_language(llm, result, question):
    # Construct a prompt for the LLM
    prompt = f"Transform the following result into a natural language statement:\n\nResult: {result}, where the user question was : {question} \n\nOutput:"
    
    # Call the LLM to generate the response
    response = llm_new(prompt)
    return response

In [16]:
# Example usage
natural_language_response = transform_to_natural_language(llm_new, execute_cypher_query(graph, cypher_query), question)
print(natural_language_response)

/Users/dhruvchandel/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Transform the following result into a natural language statement:

Result: [{'numberOfArticles': 30}], where the user question was : How many published papers are there? 

Output: There are 30 published papers.
